In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
class KalmanFilter(object):
    def __init__(self, A = None, C = None, Gamma = None, Sigma = None, P = None, u0 = None, V0 = None,x1=None,n=None):

        if(A is None or C is None):
            raise ValueError("Set proper system dynamics.")

        self.k = A.shape[1] # number of hidden states
        self.n = n # number of observations

        self.A = A # A is thr transition probability matrix
        self.C = C # C is the emission probability matrix
        
        self.Gamma = np.eye(self.k) if Gamma is None else Gamma # Gamma is the covariance matrix of noise term added to the hidden state transition
        self.Sigma = np.eye(self.k) if Sigma is None else Sigma # Sigma is the covariance matrix of noise term added to the emission 
        self.P = np.eye(self.k) if P is None else P # P is an intermediate variable during inference

        self.u = np.zeros(self.n,self.k)
        self.V = np.zeros(self.n,self.k)
        self.K = np.zeros(self.n,self.k)
        self.c = np.zeros(self.n,self.k)

        self.u0 = u0 # u0 is the initial estimate of the mean of z1
        self.V0 = V0 # V0 is the initial estimate of the variance of z1
        S = np.dot(np.dot(self.C,self.V0),self.C.T) + self.Sigma
        T = np.dot(self.C,self.u0)
        I = np.eye(self.k)

        self.K[0] = np.dot(np.dot(self.V0, self.C.T), np.linalg.inv(S))

        self.u[0] = self.u0 + np.dot(self.K,x1-np.dot(self.C,self.u0))
        self.V[0] = np.dot((I- np.dot(self.K,self.C)),self.V0)
        self.c[0] = norm(x1,T,S)
    
    def forward(self,x):
        # during inference, u[n], V[n], c[n] are calculated
        self.P = np.dot(np.dot(self.A,self.V),self.A.T) + self.Gamma
        I = np.eye(self.n)
        S = np.dot(np.dot(self.C,self.P),self.C.T) + self.Sigma
        T = np.dot(np.dot(self.C,self.A),self.u)
        
        self.K = np.dot(np.dot(self.P,self.C.T),np.linalg.inv(S))
        self.u = np.dot(self.A,self.u) + np.dot(self.K, x-T)
        self.V = np.dot((I-np.dot(self.K,self.C)),self.P)
        self.c = norm(x,T,S)

    def backward(self):
        

    def learning(self, z):
        y = z - np.dot(self.C, self.z)
        
        S = self.Sigma + np.dot(self.C, np.dot(self.P, self.C.T))
        K = np.dot(np.dot(self.P, self.C.T), np.linalg.inv(S))
        self.z = self.z + np.dot(K, y)

        I = np.eye(self.n)
        # self.P = np.dot(I - np.dot(K, self.C), self.P)
        self.P = np.dot(np.dot(I - np.dot(K, self.C), self.P), 
        	(I - np.dot(K, self.C)).T) + np.dot(np.dot(K, self.Sigma), K.T)



In [ ]:
def example():
	dt = 1.0/60
	A = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
	C = np.array([1, 0, 0]).reshape(1, 3)
	Gamma = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
	Sigma = np.array([0.5]).reshape(1, 1)

	x = np.linspace(-10, 10, 100)
	measurements = - (x**2 + 2*x - 2)  + np.random.normal(0, 2, 100)

	kf = KalmanFilter(A = A, C = C, Gamma = Gamma, Sigma = Sigma)
	predictions = []

	for z in measurements:
		predictions.append(np.dot(C,  kf.predict())[0])
		kf.update(z)

	plt.plot(range(len(measurements)), measurements, label = 'Measurements')
	plt.plot(range(len(predictions)), np.array(predictions), label = 'Kalman Filter Prediction')
	plt.legend()
	plt.show()

if __name__ == '__main__':
    example()